*Disclaimer*: This notebook borrows from different sources including https://en.wikipedia.org/wiki/Deutsch–Jozsa_algorithm#Deutsch's_algorithm, https://learn.qiskit.org/course/algorithms/phase-kickback, and https://learn.qiskit.org/course/ch-algorithms/deutsch-jozsa-algorithm.

# Deutsch's problem

<img src="images/deutsch-problem.svg" style="float:right" />

Deutsch's problem is a special case of Deutsch-Jozsa algorithm and provides an artificial problem where we are dealing with a black box function $f : \{0, 1\} \rightarrow \{0, 1\}$ that acts on two bits, $a, b$. The function will not change bit $a$ but may or may not flip the bit $b$. We are asked to work out whether $f$ behaves differently depending on the value of $a$ (we call this a *balanced* behaviour) or if it ignores $a$ and always does the same thing to $b$ (*constant* behaviour).

The challenge is to apply $f$ as few times as possible.

The best classical algorithm we can imagine here is to call $f$ twice to check the two different options for $a$ and see how $f$ behaves.

# Deutsch's algorithm

There are four possible functions we need to consider for *constant* ($f_i(0)=f_i(1)$) and *balanced* ($f_i(0) \neq f_i(1)$) behaviour:

| Function $f_i$ | x = 0 | x = 1 | $f_i(0) \oplus f_i(1)$ | Behaviour |
|----------------|-------|-------|------------------------|-----------|
| $f_0$          | $0$   | $0$   | $0$                    | constant  |
| $f_1$          | $0$   | $1$   | $1$                    | balanced  |
| $f_2$          | $1$   | $0$   | $1$                    | balanced  |
| $f_3$          | $1$   | $1$   | $0$                    | constant  |

We can create a quantum algorithm that does even better than the classical solution by only calling $f$ once.
The algorithm one can construct first puts the qubit $a$ into the superposition state $|+\rangle$ (to check each value for $a$), and qubit $b$ into the $|-\rangle$ state. Any flip conditioned on $a$ will result in an effect called *phase kickback*, flipping qubit $a$ from $|+\rangle$ to $|-\rangle$. To measure this effect in the Z-basis, we then apply a Hadamard gate and make a measurement on qubit $a$ to see whether a phase kickback occured or not.

<img src="images/deutsch-algorithm.svg" style="float:left" />

# Running on Qiskit

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

In [ ]:
def deutsch_problem(seed=None) -> QuantumCircuit:
    """Returns a circuit that can act as a black box for 
    Deutsch's algorithm.
    
    returns: QuantumCircuit
    """
    np.random.seed(seed)
    problem = QuantumCircuit(2)
    if np.random.randint(2):
        print("Function is balanced")
        problem.cx(0, 1)
    else:
        print("Function is constant")
    if np.random.randint(2):
        problem.x(1)
    return problem

In [ ]:
def deutsch(problem: QuantumCircuit) -> QuantumCircuit:
    # Add code here and return the resulting quantum circuit
    return circuit

In [ ]:
circuit = deutsch(deutsch_problem())
circuit.draw("mpl")

In [ ]:
simulator = QasmSimulator()
# Exectue the circuit on qasm simulator
job = simulator.run(circuit, shots=1000)
result = job.result()
counts = result.get_counts(circuit)

The `QasmSimulator` backend is designed to mimic an actual device. It executes a Qiskit QuantumCircuit and returns a dictionary containing the final values of the classical registers in the circuit.

In [ ]:
print(f"Measured a {'constant' if '0' in counts else 'balanced'} circuit ({counts})")
plot_histogram(counts)

# Doing the math

First, our black box – a so-called oracle – can be expressed as a unitary operator $U_f|x\rangle|y\rangle = |x\rangle|y\otimes f(x)\rangle$.

1. The qubits are initally in state $|00\rangle$. So we apply an X-gate to get in the $|01\rangle$ qubit state.

$$|\psi_0\rangle = |0\rangle X|0\rangle = |0\rangle |1\rangle = |01\rangle$$

2. We then apply a hadamard gate on both qubits

$$
\begin{align*}
|\psi_1\rangle &= (H \otimes H)|\psi_0\rangle = H^{\otimes 2}|01\rangle =\frac{1}{2}(|0\rangle + |1\rangle)(|0\rangle - |1\rangle \\
&= \frac{1}{2}|0\rangle(|0\rangle-|1\rangle) + \frac{1}{2}|1\rangle(|0\rangle-|1\rangle)
\end{align*}
$$

3. Next we apply the oracle gate $U_f$

$$
\begin{align*}
|\psi_2\rangle = U_f|\psi_1\rangle = \frac{1}{2}|0\rangle(|0 \oplus f(0)\rangle - |1 \oplus f(0)\rangle) + \frac{1}{2}|1\rangle(|0 \oplus f(1)\rangle - |1 \oplus f(1)\rangle)
\end{align*}
$$

- When looking closely we see that $|0\oplus a\rangle - |1 \oplus a\rangle = (-1)^a(|0\rangle - |1\rangle)$
  
$$
\begin{align*}
|\psi_2\rangle &= \frac{1}{2}(-1)^{f(0)}|0\rangle(|0\rangle - |1\rangle) + \frac{1}{2}(-1)^{f(1)}|1\rangle(|0\rangle - |1\rangle) \\
&= \left(\frac{1}{\sqrt{2}} (-1)^{f(0)}|0\rangle + (-1)^{f(1)}|1\rangle\right)\left(\frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)\right)\\
&= (-1)^{f(0)} \left(\frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}(-1)^{f(0)\oplus f(1)}|1\rangle\right) |-\rangle
\end{align*}
$$

Note: As discussed in the last lecture, the global phase might be ignored here.

From now on, one can ignore the second qubit for simplicity. However, we go on with the full state.

## So first let's consider the *constant* behaviour.

- If $U_f$ is *constant* then $f(0) \oplus f(1) = 0$

$$
\begin{align*}
|\psi_2\rangle &= (-1)^{f(0)} \left(\frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}(-1)^0|1\rangle\right) |-\rangle \\
&= (-1)^{f(0)} \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) |-\rangle
\end{align*}
$$

4. Finally we apply the hadamard gate to the first qubit

$$
\begin{align*}
|\psi_3\rangle &= H|\psi_2\rangle = (-1)^{f(0)} H\left(\frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}|1\rangle\right) |-\rangle \\
&= (-1)^{f(0)} |0\rangle|-\rangle
\end{align*}
$$

The probability of measuring $|0\rangle$ in the first qubit is 1. This means that for a *constant* function, a measurement of the first qubit is certain to return $|0\rangle$.

## Now let's consider the *balanced* behaviour.

- If $U_f$ is *balanced* then $f(0) \oplus f(1) = 1$

$$
\begin{align*}
|\psi_2\rangle &= (-1)^{f(0)} \left(\frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}(-1)^1|1\rangle\right) |-\rangle \\
&= (-1)^{f(0)} \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle) |-\rangle
\end{align*}
$$

- Finally we apply the hadamard gate to the first qubit

$$
\begin{align*}
|\psi_3\rangle &= H|\psi_2\rangle = (-1)^{f(0)} H\left(\frac{1}{\sqrt{2}}|0\rangle - \frac{1}{\sqrt{2}}|1\rangle\right) |-\rangle \\
&= (-1)^{f(0)} |1\rangle|-\rangle
\end{align*}
$$

The probability of measuring $|1\rangle$ in the first qubit is 1. This means that for a *balanced* function, a measurement of the first qubit is certain to return $|1\rangle$.

# Generalizing Deutsch's algorithm to Deutsch-Jozsa

![](images/deutsch_jozsa-algorithm.png)


1. Prepare the first qubits in the zero state and the last in the one state

$$ |\psi_0\rangle = |0\rangle^{\otimes n}|1\rangle$$

2. Apply a hadamard gate to each qubit

$$ |\psi_1\rangle = H^{\otimes n}|\psi_0\rangle = \frac{1}{\sqrt{2^n+1}} \sum_{x=0}^{2^n-1}|x\rangle(|0\rangle - |1\rangle)$$

3. Apply the oracle $U_f$, that maps $|x\rangle|y\rangle$ to $|x\rangle|y \oplus f(x)\rangle$

$$
\begin{align*}
|\psi_2\rangle &= U_f|\psi_1\rangle \\
&= \frac{1}{\sqrt{2^n+1}} \sum_{x=0}^{2^n-1}|x\rangle (|(f(x)\rangle - |1 \oplus f(x)\rangle) \\
&= \frac{1}{\sqrt{2^n+1}} \sum_{x=0}^{2^n-1} (-1)^{f(x)}|x\rangle(|0\rangle - |1\rangle)
\end{align*}
$$

4. We now ignore the second qubit

$$
\begin{align*}
|\psi_3\rangle &= \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} (-1)^{f(x)}|x\rangle
\end{align*}
$$

5. ... and apply hadarmard gates to the first qubit register

$$
\begin{align*}
|\psi_4\rangle &= H^{\otimes n}|\psi_3\rangle \\
&= \frac{1}{2^n} \sum_{x=0}^{2^n-1} (-1)^{f(x)} \left[ \sum_{y=0}^{2^n-1} (-1)^{x \cdot y} |y\rangle \right] \\
&= \frac{1}{2^n} \sum_{y=0}^{2^n-1} \left[ \sum_{x=0}^{2^n-1} (-1)^{f(x)}(-1)^{x\cdot y} \right] |y\rangle
\end{align*}
$$

$x\cdot y$ is the sum of the bitwise product $x_0 y_0 \oplus x_1 y_1 \oplus \dots \oplus x_{n-1} y_{n-1}$.

6. Measure the first register. In case the all-zero state is measured ($|0\rangle^{\otimes n} = |\frac{1}{2^n}\sum_{x=0}^{2^n-1}(-1)^{f(x)}|^2$) we have a constant function. If the probability for measuring all-zero is 0, we have a balanced function.

# Optional Homework Exercise
Create a quantum circuit for the Deutsch-Jozsa algorithm that uses an oracle that takes two qubits as input. Using an ancillary/auxillary qubit (i.e. three qubits in total) your code should do the following

- Process the three qubits in a specific manner befrore they reach the oracle $U_f$
- Process the qubits after the oracle
- Measure the non-ancillary qubits and determine whether the oracle is constant or balanced

In [ ]:
from typing import List

def oracle(numbers: List[int]) -> QuantumCircuit:
    """Returns a circuit that can act as a oracle for 
    Deutsch-Jozsa algorithm.
    
    returns QuantumCircuit
    """
    circuit = QuantumCircuit(3)
    assert(all(number <= 1 for number in numbers))
    for i in numbers:
        circuit.cx(i, 2)
    return circuit

In [ ]:
import qiskit.tools.jupyter
print("This notebook was created and tested with the following version numbers")
%qiskit_version_table